# load data

In [ ]:
from data import store, info
store

In [ ]:
train=store['train']
train.head()

# load raw features

In [ ]:
f_user = store['feat-user-default']
f_user.head()

In [ ]:
f_ad = store['feat-ad-default']
f_ad.head()

In [ ]:
f_pos = store['position']
f_pos.head()

# experiment by your self to design a extractor 

In [ ]:
train = store['train']
test = store['test']
train_feats = train.columns
test_feats = test.columns
label_name = 'label'
print train_feats
print test_feats
print label_name

In [ ]:
common_feat = list(set(train_feats).intersection(set(test_feats)))
common_feat.remove(label_name)
print common_feat

class RawData(object):
    pass
raw_data = RawData()

raw_data.store = store
raw_data.label_name = label_name
raw_data.common_feat = common_feat

In [ ]:
#%%writefile competition/featExtract/simple_features.py

from competition.featExtract.base import Extractor
import pandas as pd



class SimpleFeatures(Extractor):
    def __init__(self, raw_data):
        self.raw_data = raw_data
        store = raw_data.store
        f_user = store['feat-user-default']
        f_ad = store['feat-ad-default']
        f_pos = store['position']
        def merge_tables(X):
            # user
            X = pd.merge(X,f_user,how='left',left_on='userID',right_on='userID')
            del X['userID']
            # ad
            X = pd.merge(X,f_ad,how='left',left_on='creativeID',right_on='creativeID')
            del X['creativeID']
            # pos
            X = pd.merge(X,f_pos,how='left',left_on='positionID',right_on='positionID')
            del X['positionID']

            return X
        self.merge_tables = merge_tables

    def get_train(self):
        X,y = self._get(self.raw_data.store['train']) 
        X = self.merge_tables(X)
        return X,y

    def get_test(self,dataframe):
        X,y = self._get(self.raw_data.store['test']) 
        X = self.merge_tables(X)
        return X,y

    def _get(self,dataframe):
        X = dataframe[self.raw_data.common_feat]
        y = dataframe[self.raw_data.label_name]
        return X,y


In [ ]:
from competition.featExtract.simple_features import SimpleFeatures
prep = SimpleFeatures(raw_data)

trainX,trainY = prep.get_train()
testX,testY  = prep.get_train()


In [ ]:
print trainX.shape
print testX.shape

# run the extractor
- modify `config.py`
- run `extract_features.py`

In [ ]:
!cat config.py

In [ ]:
%run extract_features.py

In [ ]:
!ls ./_extracted/